# Исследование надежности заемщиков.

---



❗ *Оставлены рекомендации проверившего проект ревьюера*

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Также помимо главной цели исследования попробуем ответить на вопросы:
* Как разные цели кредита влияют на его возврат в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Есть ли зависимость между количеством детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Все данные собраны в таблицу и подписаны следующим образом:

*children* — количество детей в семье;    
*days_employed* — общий трудовой стаж в днях;    
*dob_years* — возраст клиента в годах;    
*education* — уровень образования клиента;    
*education_id* — идентификатор уровня образования;    
*family_status* — семейное положение;    
*family_status_id* — идентификатор семейного положения;    
*gender* — пол клиента;    
*income_type* — тип занятости;    
*debt* — имел ли задолженность по возврату кредитов;    
*total_income* — ежемесячный доход;    
*purpose* — цель получения кредита.

### Содержание

* [Шаг 1. Обзор данных](#1)  
* [Шаг 2. Предобработка данных](#2)  
  * [Шаг 2.1 Заполнение пропусков](#2.1)
  * [Шаг 2.2 Удаление дубликатов](#2.2)
  * [Шаг 2.3 Изменение типов данных](#2.3)
  * [Шаг 2.4 Проверка данных на аномалии и исправления](#2.4)
  * [Шаг 2.5 Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма](#2.5)
  * [Шаг 2.6 Категоризация дохода](#2.6)
  * [Шаг 2.7 Категоризация целей кредита](#2.7)
* [Шаг 3. Ответы на вопросы](#3)
  * [Есть ли зависимость между количеством детей и возвратом кредита в срок?](#3.1)
  * [Есть ли зависимость между семейным положением и возвратом кредита в срок?](#3.2)
  * [Есть ли зависимость между уровнем дохода и возвратом кредита в срок?](#3.3)
  * [Как разные цели кредита влияют на его возврат в срок?](#3.4)
* [Шаг 4. Общий вывод](#4)

<a id='1'></a>

### Шаг 1. Обзор данных

Импортируем нужные нам для работы библиотеки и читаем файл.


In [ ]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()

df = pd.read_csv('/datasets/data.csv')

Выведем первые 10 строк, чтобы посмотреть, как сейчас выглядит таблица.

In [ ]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Также сразу посмотрим на типы данных методом info() и воспользуемся методом describe(), чтобы посмотреть средние, максимальные и минимальные значения.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [ ]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


<a id='2'></a>

**Что видим:**  
  
1. В столбцах *days_employed* и *total_income* есть пропущенные значения. Их равное количество, есть подозрения, что они в одинаковых строках.
2. Минимальное количество детей -1. Такого быть не может: или дети есть или их нет.
3. Максимальное количество детей 20. Это может быть правдой, а может быть и опечаткой.
4. Отрицательное количество дней трудоустройства, такого быть не может.
5. У столбца *total_income* тип данных float64, поэтому он так нечитаемо записан.
6. Среднее значение в столбце *days_employed* 63046 дней, это почти 173 года. А максимально кто-то отработал 1100 лет. Такого быть не может, в данных есть выбросы.
7. Минимальный возраст — 0. Это не было бы странным, если бы мы не говорили о выдаче кредита, банки обязательно запрашивают возраст. Видимо какие-то данные были потеряны.

### Шаг 2. Предобработка данных

<a id='2.1'></a>

### Шаг 2.1 Заполнение пропусков

In [ ]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Начнём с заполнения пропусков. Как мы уже выяснили, явные пропущенные значения есть в столбцах days_employed и total_income.  
Проверим такую гипотезу:  
**Данные в столбце days_employed пропущены в тех же строках, как и данные в total_income.**

Это поможет нам определиться, каким образом будем пропуски заполнять.  

Для того, чтобы её проверить, посчитаем количество строк, где в обоих столбцах пропущены значения. Если их будет 2174, то гипотеза подтвердится.

In [ ]:
df[(df['days_employed'].isna()) & (df['total_income'].isna())].shape

(2174, 12)

<div class="alert alert-success"; style="border-left: 7px solid green">
<h3>✔ Комментарий ревьюера</h3>
    
Можно проверить как `(df['days_employed'].isna() == (df['total_income'].isna()).all()`</div>    


Гипотеза подтвердилась.

Избавляться от строк с пропусками мы не будем, так как потеряем большой набор данных, будем пропуски заполнять.

Сгруппируем данные по типу занятости и заполним пропуски медианами. Среднее здесь не подойдёт, потому что у нас есть выбросы, картина не будет соответствовать действительности.  

Начнём с доходов. Я сделаю отдельную табличку с медианными доходами, сгруппированными по типу занятости.

In [ ]:
median_income = df.groupby(['income_type']).agg({'total_income' : 'median'})
median_income

,total_income
income_type,
безработный,131339.751676
в декрете,53829.130729
госслужащий,150447.935283
компаньон,172357.950966
пенсионер,118514.486412
предприниматель,499163.144947
сотрудник,142594.396847
студент,98201.625314


Напишем цикл, который заменит все пропуски медианными значениями из таблицы.


In [ ]:
for income_type in median_income.index:
    medians_to_fill = median_income.loc[income_type, 'total_income']
    df.loc[df['income_type'] == income_type, 'total_income'] = df.loc[df['income_type'] == income_type, 'total_income'].fillna(medians_to_fill)

#проверка на пропуски
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

<div class="alert alert-success"; style="border-left: 7px solid green">
<h3>✔ Комментарий ревьюера</h3>
    
Хороший способ, можно такое делать как
    
```python
df['total_income'] = df['total_income'].\
                                    fillna(df.groupby('income_type')['total_income'].\
                                    transform(lambda x : x.median()))
    
```

Напишу тут также пару общих слов, на которые при замене стоит обращать внимание. Такая замена работает, только если пропуски распределены случайно, т.е. вероятность пропуска в строке не зависит от других значений в строке. Что если бы все прпоуски в столбце `total_income` были только у людей без образования, а у тех людей без образования, где пропусков нет, доход указан 0? Тогда надо было бы заменить пропуски на 0.
    
Проверять такое можно по-разному, можно, например, для каждого столбца сравнить распределение значения для среза с прпоусками в `total_income` и среза без пропусков. Сделать это можно через `value_counts(normalize=True)`, либо можно группировать по каждому столбцу и смотреть долю пропусков в каждой группе, если их одинаково пропуски распределены случайно (по крайней мере относительно выбранного столбца).
    
Подробнее об этом можно почитать тут https://loginom.ru/blog/missing  или посмотреть видео: https://youtu.be/YpqUbirqFxQ

</div>    


Теперь разберёмся с отработанными днями. Но перед этим избавимся от отрицательных значений в столбце. Умножение на -1 не подойдёт, потому что есть и положительные, поэтому нам нужно взять значения по модулю.

In [ ]:
df['days_employed'] = df['days_employed'].abs()

df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.538908,66914.728907,43.293380,0.817236,0.972544,0.080883,1.652253e+05
std,1.381587,139030.880527,12.574584,0.548138,1.420324,0.272661,9.804367e+04
min,-1.000000,24.141633,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,927.009265,33.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,2194.220567,42.000000,1.000000,0.000000,0.000000,1.425944e+05
75%,1.000000,5537.882441,53.000000,1.000000,1.000000,0.000000,1.955499e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Теперь сделаем такую же табличку с медианными значениями, которыми будем заполнять пропуски по тому же принципу, как и с доходом. Среднее здесь тоже, мне кажется, не подойдёт для заполнения, так как точно есть выбросы плюс стаж у разных людей может сильно различаться.

In [ ]:
median_days = df.groupby(['income_type']).agg({'days_employed' : 'median'})
median_days

,days_employed
income_type,
безработный,366413.652744
в декрете,3296.759962
госслужащий,2689.368353
компаньон,1547.382223
пенсионер,365213.306266
предприниматель,520.848083
сотрудник,1574.202821
студент,578.751554


Можно заметить, что безработные и пенсионеры отработали просто дикое количество дней.
  

Если медиана показала такое страшное количество, значит, точно от половины безработных и пенсионеров придётся избавиться, что может повлиять на результаты. Поэтому пока удалять их не буду.  
Сначала посмотрю минимальный стаж по группе.

In [ ]:
df[df['income_type'] == 'безработный']['days_employed'].min()

337524.4668348471

In [ ]:
df[df['income_type'] == 'пенсионер']['days_employed'].min()

328728.72060451825

Не избавляться от данных сразу было хорошим решением: тогда бы нам пришлось потерять полностью две группы заёмщиков.  
  
  
Сделаем так: безработным мы полностью обнулим стаж. Какая разница, сколько ты отработал, если не момент взятия кредита ты был безработным и не имел дохода.  
  
С пенсионерами поступим немного по-другому: чтобы в России стать пенсионером, нужно отработать не менее 15 лет. Поменяем всем пенисонерам стаж на минимальный — в этом исследовании точное количество отработанных лет нам не понадобится.

In [ ]:
df.loc[df['income_type'] == 'безработный', 'days_employed'] = 0

In [ ]:
df.loc[df['income_type'] == 'пенсионер', 'days_employed'] = 5475

Теперь можно заполнить пропуски медианами

In [ ]:
for income_type in median_days.index:
    medians_to_fill = median_days.loc[income_type, 'days_employed']
    df.loc[df['income_type'] == income_type, 'days_employed'] = df.loc[df['income_type'] == income_type, 'days_employed'].fillna(medians_to_fill)

#проверка на пропуски
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Пропусков больше нет.    
Мне кажется, что природа появления этих данных — ручная или автоматическая ошибка при вычислении точного количества дней: вышла промашка с запятой, отделяющей дробную часть.  
  
Можно приступать к удалению дубликатов.

<a id='2.2'></a>

### Шаг 2.2 Удаление дубликатов.

Приведу значения в столбце education к нижнему регистру, чтобы не пропустить дубликаты. Посчитаем количество дубликатов методом «duplicated()»

In [ ]:
df['education'] = df['education'].str.lower()
df.duplicated().sum()

71

Это 0.3% данных, можем смело от них избавиться

In [ ]:
df = df.drop_duplicates().reset_index(drop = True)
df.duplicated().sum()

0

Дубликатов в датафрейме больше нет  
  
Думаю, что причиной их появления может быть ошибочное создание дублей при ручном внесении данных.

<a id='2.3'></a>

### Шаг 2.3. Изменение типов данных.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  float64
 11  purpose           21454 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Нам сейчас не особо важна точность значений ежемесячного дохода и общего трудового стажа, поэтому переведём вещественный тип данных этих столбцов в целочисленный. Так будет и нагляднее, и заодно сократится время обработки данных

In [ ]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21454 non-null  int64 
 1   days_employed     21454 non-null  int64 
 2   dob_years         21454 non-null  int64 
 3   education         21454 non-null  object
 4   education_id      21454 non-null  int64 
 5   family_status     21454 non-null  object
 6   family_status_id  21454 non-null  int64 
 7   gender            21454 non-null  object
 8   income_type       21454 non-null  object
 9   debt              21454 non-null  int64 
 10  total_income      21454 non-null  int64 
 11  purpose           21454 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


<a id='2.4'></a>

### Шаг 2.4. Проверка данных на аномалии и исправления.

Аномально большой и отрицательный трудовой стаж я уже обработала при заполнении пропусков, вернёмся к выводам из пункта обзора данных и вспомним, что ещё осталось без внимания.  

1. Минимальное количество детей -1.
2. Максимальное количество детей 20.
3. Минимальный возраст — 0.  
  
Начнём с минимального возраста. Кредит нельзя взять до совершеннолетия, однако могут бысть исключения, когда лицо могут признать дееспособным ещё до 18 лет. Но это очень редкие случаи, да и сейчас у нас нет возможности проверить каждого такого человека отдельно, поэтому этим пренебрежём и сделаем всех нулевых заёмщиков 18-ти летними.

In [ ]:
df.loc[df['dob_years'] == 0, 'dob_years'] = 18
df['dob_years'].min()

18

<div class="alert alert-success"; style="border-left: 7px solid green">
<h3>✔ Комментарий ревьюера</h3>
    
Если такие значения распределены равномерно по датасету, то их можно заменять медианой
</div>    


Проверим на всякий случай, нет ли у нас таких пенсионеров

In [ ]:
df[(df['income_type'] == 'пенсионер') & (df['dob_years'] == 18)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,5475,18,среднее,1,женат / замужем,0,F,пенсионер,0,71291,автомобиль
578,0,5475,18,среднее,1,женат / замужем,0,F,пенсионер,0,97620,строительство собственной недвижимости
1175,0,5475,18,среднее,1,женат / замужем,0,F,пенсионер,0,313949,получение дополнительного образования
1898,0,5475,18,среднее,1,вдовец / вдова,2,F,пенсионер,0,127400,на покупку автомобиля
2284,0,5475,18,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514,недвижимость
4918,0,5475,18,высшее,0,вдовец / вдова,2,F,пенсионер,1,183556,свой автомобиль
6405,0,5475,18,высшее,0,гражданский брак,1,F,пенсионер,0,118514,свадьба
6664,0,5475,18,высшее,0,в разводе,3,F,пенсионер,0,118514,покупка жилой недвижимости
7028,0,5475,18,высшее,0,Не женат / не замужем,4,F,пенсионер,0,263121,образование
8052,0,5475,18,высшее,0,Не женат / не замужем,4,F,пенсионер,0,61804,высшее образование


О нет, есть :(
Тогда меняем тактику, всем 18-ти летним пенсионерам заменим возвраст на медианный

In [ ]:
df.loc[(df['income_type'] == 'пенсионер') & (df['dob_years'] == 18), 'dob_years'] = df[df['income_type'] == 'пенсионер']['dob_years'].median()
df[(df['income_type'] == 'пенсионер') & (df['dob_years'] == 18)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Теперь можно переходить к детям.
  
Сложно опечататься и написать вместо 1 отрицательное число, так что будем считать , что это машинныая ошибка и заменим всех отрицательных детей на нули.

In [ ]:
df['children'] = df['children'].replace(-1, 0)
df[df['children'] == -1].count()[0]

0

Посчитаем, сколько людей с 20 детьми, чтобы понять, выбросы ли это или всё же есть один счастливчик (в одного ещё можно поверить, в 2 и более уже тяжело)

In [ ]:
df[df['children'] == 20].count()[0]

76

Целых 76 штук, это очевидно выбросы. На стандартной клавиатуре 2 и 0 стоят в опасной близости, так что будем считать это опечатками и заменим значения на нули

In [ ]:
df.loc[df['children'] == 20, 'children'] = 2
df[df['children'] == 20].count()[0]

0

<a id='2.5'></a>

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим два новых датафрейма, в которых:
1. Каждому уникальному значению из education соответствует уникальное значение education_id — в первом;
2. Каждому уникальному значению из family_status соответствует уникальное значение family_status_id — во втором.

А из исходного датафрейма удалим столбцы education и family_status, они нам будут больше не нужны, так как новые таблицы станут словарями, к которым мы сможем обращаться, если потребуется

In [ ]:
education = df.pivot_table(index='education', values='education_id')

education

,education_id
education,
высшее,0
начальное,3
неоконченное высшее,2
среднее,1
ученая степень,4


<div class="alert alert-success"; style="border-left: 7px solid green">
<h3>✔ Комментарий ревьюера</h3>
    
Решение неплохое через `pivot`, можно также `df[['education','education_id']].drop_duplicates().reset_index()`. В индексах обычно должны всё же `id` стоять :) Также такой `pivot` среднее считает, можно сперва проверить, нет ли у одного `education` два разных `id`
</div>    


In [ ]:
family_status = df.pivot_table(index='family_status', values='family_status_id')

family_status

,family_status_id
family_status,
Не женат / не замужем,4
в разводе,3
вдовец / вдова,2
гражданский брак,1
женат / замужем,0


In [ ]:
df.drop(columns=['education', 'family_status'])
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,5475,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


<a id='2.6'></a>

### Шаг 2.6. Категоризация дохода.

Создадим столбец total_income_category с категориями:

    0–30000 — 'E';
    30001–50000 — 'D';
    50001–200000 — 'C';
    200001–1000000 — 'B';
    1000001 и выше — 'A'.

In [ ]:
def income_category(row):

    if row <= 30000:
        return 'E'

    elif  (row >= 30001) and (row <= 50000):
        return 'D'

    elif  (row >= 50001) and (row <= 200000):
        return 'C'

    elif (row >= 200001) and (row <= 1000000):
        return 'B'

    return 'A'

In [ ]:
df['total_income_category'] = df['total_income'].apply(income_category)

df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,5475,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,926,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B
6,0,2879,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B
7,0,152,50.0,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C
8,2,6929,35.0,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,2188,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C


Теперь от столбца с доходом можно избавиться

In [ ]:
df.pop('total_income')
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,purpose,total_income_category
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,покупка жилья,B
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,приобретение автомобиля,C
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,покупка жилья,C
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,дополнительное образование,B
4,0,5475,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,сыграть свадьбу,C


<a id='2.7'></a>

### Шаг 2.7. Категоризация целей кредита.

Мне заранее известно, на какие категории нужно делить данные:

    'операции с автомобилем',
    'операции с недвижимостью',
    'проведение свадьбы',
    'получение образования'
    
Сначала посмотрим, какие уникальные цели у нас вообще есть.

In [ ]:
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Я очень ленивая и мне так не хочется вручную прописывать уникальные цели в условие при категоризации, пусть их и не много. Поэтому я воспользуюсь лемматизацией (приведение слов к первоначальной словарной форме (лемме)):

1. Если в цели кредита в столбце будут слова с леммой «автомобиль», то такие цели будут иметь категорию «операции с автомобилем».
2. Если будут слова с леммой «недвижимость», «жилье» то такие цели перейдут в категорию «операцции с недвижимостью».
3. Леммы «свадьба» — категория «проведение свадьбы».
4. Леммы «образование» — категория «получение образования»

Если бы у нас было очень много уникальных целей, по которым было бы тяжело пробежаться глазами и выделить ключевые слова, то синонимов было бы больше.

<div class="alert alert-success"; style="border-left: 7px solid green">
<h3>✔ Комментарий ревьюера</h3>
    
Можно без лемматизации, проверяя просто основу слова, т.е. `if 'свадь' in purpose: ...` и т.п.</div>    


In [ ]:
def purpose_category(row):

    lemmas = m.lemmatize(row)

    if ('недвижимость' in lemmas) or ('жилье' in lemmas):
        return 'операции с недвижимостью'

    elif 'образование' in lemmas:
        return 'получение образования'

    elif 'автомобиль' in lemmas:
        return 'операции с автомобилем'

    elif 'свадьба' in lemmas:
        return 'проведение свадьбы'


In [ ]:
df['purpose_category'] = df['purpose'].apply(purpose_category)
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,purpose,total_income_category,purpose_category
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,покупка жилья,B,операции с недвижимостью
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,покупка жилья,C,операции с недвижимостью
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,дополнительное образование,B,получение образования
4,0,5475,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,сыграть свадьбу,C,проведение свадьбы
5,0,926,27.0,высшее,0,гражданский брак,1,M,компаньон,0,покупка жилья,B,операции с недвижимостью
6,0,2879,43.0,высшее,0,женат / замужем,0,F,компаньон,0,операции с жильем,B,операции с недвижимостью
7,0,152,50.0,среднее,1,женат / замужем,0,M,сотрудник,0,образование,C,получение образования
8,2,6929,35.0,высшее,0,гражданский брак,1,F,сотрудник,0,на проведение свадьбы,C,проведение свадьбы
9,0,2188,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,покупка жилья для семьи,C,операции с недвижимостью


Можем избавляться от столбца с целями, категорий будет достаточно

In [ ]:
df.pop('purpose')
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income_category,purpose_category
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,B,операции с недвижимостью
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,C,операции с автомобилем
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,C,операции с недвижимостью
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,B,получение образования
4,0,5475,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,C,проведение свадьбы


<a id='3'></a>

### Шаг 3. Ответы на вопросы.

<a id='3.1'></a>

##### Вопрос 1:

**Есть ли зависимость между количеством детей и возвратом кредита в срок?**

Чтобы ответить на этот вопрос, данные снова нужно категоризировать :)

В России многодетной семьёй считается семья, где 3 и более детей. Поэтому по количеству детей разобью на категории:

    «нет детей»;
    «1-2 ребенка»;
    «многодетные»(>= 3 детей).

In [ ]:
def children_category(row):
    if row == 0:
        return 'нет детей'
    elif 1 <= row <= 2:
        return '1-2 ребенка'
    return 'многодетные'

In [ ]:
df['children_category'] = df['children'].apply(children_category)

df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income_category,purpose_category,children_category
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,B,операции с недвижимостью,1-2 ребенка
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,C,операции с автомобилем,1-2 ребенка
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,C,операции с недвижимостью,нет детей
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,B,получение образования,многодетные
4,0,5475,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,C,проведение свадьбы,нет детей
5,0,926,27.0,высшее,0,гражданский брак,1,M,компаньон,0,B,операции с недвижимостью,нет детей
6,0,2879,43.0,высшее,0,женат / замужем,0,F,компаньон,0,B,операции с недвижимостью,нет детей
7,0,152,50.0,среднее,1,женат / замужем,0,M,сотрудник,0,C,получение образования,нет детей
8,2,6929,35.0,высшее,0,гражданский брак,1,F,сотрудник,0,C,проведение свадьбы,1-2 ребенка
9,0,2188,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,C,операции с недвижимостью,нет детей


Теперь сделаю таблицу, где отражу количество людей из разных категорий, которые просрочили платеж и среднее по просрочившим.

In [ ]:
children_pivot = df.pivot_table(index=['children_category'], values='debt', aggfunc=['sum', 'mean'])
children_pivot

,sum,mean
,debt,debt
children_category,,
1-2 ребенка,646,0.093137
многодетные,31,0.081579
нет детей,1064,0.075258


Многодетные и бездетные ответственнее относятся к выплатам в срок: всего 7% бездетных и 8% многодетных имеют задолженности по оплате, а доля должников с 1-2 ребёнком — 9%.

<a id='3.2'></a>

##### Вопрос 2:

**Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Для создания новой таблички смерджим исходную и словарь, не перезаписывая исходную таблицу, и достанем оттуда названия статусов

In [ ]:
famstatus_pivot = df.merge(family_status, on='family_status_id', how='left').pivot_table(index=['family_status'], values='debt', aggfunc=['sum', 'mean'])
famstatus_pivot

,sum,mean
,debt,debt
family_status,,
Не женат / не замужем,274,0.097509
в разводе,85,0.071130
вдовец / вдова,63,0.065693
гражданский брак,388,0.093471
женат / замужем,931,0.075452


Вдовцы реже всех имеют задолженность по выплатам: только 6% из взявших кредит не захотели его возвращать вовремя, а заёмщики, не скованные семейными узами — наоборот, их почти 10%.

<a id='3.3'></a>

##### Вопрос 3:

**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [ ]:
income_pivot = df.pivot_table(index=['total_income_category'], values='debt', aggfunc=['sum', 'mean'])
income_pivot

,sum,mean
,debt,debt
total_income_category,,
A,2,0.080000
B,356,0.070607
C,1360,0.084920
D,21,0.060000
E,2,0.090909


Заёмщики с очень большим доходом (категория А) и самым маленьким доходом (категория Е) чаще оплачивают кредит в срок. А вот на заёмщиков со средним доходом (категория С) приходится больше всего должников — 8.4%


<a id='3.4'></a>

##### Вопрос 4:

**Как разные цели кредита влияют на его возврат в срок?**

In [ ]:
purpose_pivot = df.pivot_table(index=['purpose_category'], values='debt', aggfunc=['sum', 'mean'])
purpose_pivot

,sum,mean
,debt,debt
purpose_category,,
операции с автомобилем,403,0.093590
операции с недвижимостью,782,0.072334
получение образования,370,0.092200
проведение свадьбы,186,0.080034


Заёмщики, берущие кредит для манипуляций с недвижимостью, наиболее ответственны — задолжников всего 7%. А вот те, кто берут кредит на автомобиль или свадьбу, не платят чаще — таких 9%.

<a id='4'></a>

## Шаг 4. Общий вывод:

1. Заёмщики, которые в прошлом были в официальном в браке или в официальном браке сейчас ответственнее, чем люди не заводившие официальные отношения. Согласно нашему исследованию только 6% вдовцов и по 7% людей в официальных отношениях и бездетных не выплачивают долг вовремя, а должников, не состоявших в официальном браке, почти 10%.  

Могу предположить, что у таких людей, во-первых, меньше инфантилизма и они чувство ответственности не только за себя, но и за партнёра, подталкивает к выполнению обязательств перед банком, а во-вторых, реже бывают проблемы с финансами: у семьи может быть один кредит, который выплачивают вдвоём, а у вдовцов же может остаться финансовая подушка после смерти партнёра.

2. Заёмщики, имеющие 1 или 2 детей - менее неответственные, чем бездетные и многодетные: их количество возрасло до 9%. Полагаю, что здесь такая картина:  
Дети — это жуткие траты, и если многодетным помогает государство, то семьи с 1-2 детьми вынуждены сами содержать своих чад, поэтому если возникает вопрос недостатка денег, то скорее просрочат кредит и испортят историю, чем сэкономят на ребёнке.


3. Банкам следует с осторожностью выдавать кредиты на устройство свадеб и образование — их возвращают гораздо реже: за свадьбу возвращают долг вовремя только 91% заёмщиков, а за обучение — 93%.

4. Зато можно не бояться выдавать кредиты клиентам с низким доходом (до 30000 денежных единиц): неплательщиков всего 7% от общего числа заёмщков с таким доходом. Преполагаю, что люди с низкими зарплатами грамотнее распределяют свои расходы и кредиты для них уже привычное дело, поэтому историю портить нежелательно. А вот люди со средним доходом (от 50001 до 200000 денежнх единиц) имеют просрочки по кредитам чаще.
